In [3]:

#path
root = '/kaggle/input/iot-modules-singleclass/iot-modules-singleclass'
train, valid, test = root+'/train/', root+'/valid/', root+'/test/'

In [4]:
#library imports
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
import cv2, os
import matplotlib.pyplot as plt

In [5]:
#test func padding
img = cv2.imread('/kaggle/input/iot-modules-singleclass/iot-modules-singleclass/test/20240610_124954_jpg.rf.87b57d00a4229f5a0438ee03914efea6.jpg')
H, W, C = img.shape
print(H, W, C)
T, B, L, R = 0, 0, 0, 0 # top bottom left right
if H > W: # cao hơn rộng => padding chiều rộng
    R = H-W
elif H < W: #rộng hơn cao => padding chiều cao
    T = W-H
else: #bang nhau
    pass
print(R, T)
transform = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Pad(padding=(L, T, R, B), fill=0, padding_mode='constant'), #trái - trên - phải - dưới
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]
)
# a = transform(img)
# print(a)
# plt.imshow(a.permute(1, 2, 0))
# plt.show()

4032 3024 3
1008 0


In [6]:
def get_paths_and_1hotvectors(path):
    file = open(path+'_classes.csv', 'r')
    DATA = file.readlines()
    label_truth = DATA[0].replace('\n', '').split(',')[1:]
    data = []
    for i in DATA[1:]:
        filename = path + i.split(',')[0]
        onehot_vector = [int(j) for j in i.split(',')[1:]]
        data.append([filename, onehot_vector])
    return label_truth, data
# x, y = get_paths_and_1hotvectors(test)
# print(x[16])
# print(x)
# print(y[0][1].index(1))
# print(y[0][0])
# print(y[0][1])
'''
Photoresistor
['7-segment-LED', 'Board-Test-830', 'Buzzer', 'Dust-sensor-GP2Y1014AU-PM2.5', 'Fingerprint-sensor-AS608-XD-65', 'IR-Remote', 'IR-receiver-1838T', 'KIT-Arduino-UNO-R3-ATMEGA16U2', 'KIT-WiFi-NodeMcu-ESP8266-CH340', 'Keypad-4x4-SMD', 'LCD-1602', 'LED', 'Led-Matrix-8x8', 'Module-Bluetooth-HC05', 'Module-Lora-RF433-SX1278-RA-01', 'Module-Matrix-8x32-MAX7219-MT-832-G', 'Photoresistor', 'Potentiometer', 'Realtime-Module-DS1307', 'Temperature-LM35', 'Ultrasonic-Range-Finder-SRF05', 'servo-motor-SG90']
16
/kaggle/input/iot-modules-singleclass/iot-modules-singleclass/test/20240610_130149_jpg.rf.064f93788c00415dff972f5ed4c5db2b.jpg
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
'''
train_ground_truth, train_img_paths_and_1hotvectors = get_paths_and_1hotvectors(train)
valid_ground_truth, valid_img_paths_and_1hotvectors = get_paths_and_1hotvectors(valid)
test_ground_truth, test_img_paths_and_1hotvectors = get_paths_and_1hotvectors(test)

In [ ]:
class IOTDataset(Dataset):
    def __init__(self, img_paths, labels, transforms=False): #label at one hot vectors
        self.img_paths = img_paths
        self.labels = labels
        self.transforms = transforms
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, index):
        path, onehotvector2intlabel = self.img_paths[index], self.labels[index].index(1)
        #img
        img_data = cv2.imread(path)
        if transforms:
            H, W, C = img_data.shape
            # print(H, W, C)
            T, B, L, R = 0, 0, 0, 0 # top bottom left right
            if H > W: # cao hơn rộng => padding chiều rộng
                R = H-W
            elif H < W: #rộng hơn cao => padding chiều cao
                T = W-H
            else: #bang nhau
                pass
            # print(R, T)
            transform = transforms.Compose(
                [
                    transforms.ToPILImage(),
                    transforms.Pad(padding=(L, T, R, B), fill=0, padding_mode='constant'), #trái - trên - phải - dưới
                    transforms.RandomRotation(degrees=5),
                    transforms.RandomHorizontalFlip(p=0.5),
                    transforms.ColorJitter(
                        brightness=0.05,
                        contrast=0.05,
                        saturation=0.05,
                        hue=0.02
                    ),

                    transforms.Resize((224, 224)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                ]
            )
            img_data_tensor = transform(img_data)
        #label
        label_tensor = torch.tensor(onehotvector2intlabel, dtype=torch.long)
        return img_data_tensor, label_tensor

#dataset
train_img_paths, train_label_vectors, valid_img_paths, valid_label_vectors, test_img_paths, test_label_vectors = [], [], [], [], [], []
for i in train_img_paths_and_1hotvectors:
    train_img_paths += [i[0]]
    train_label_vectors += [i[1]]
for i in valid_img_paths_and_1hotvectors:
    valid_img_paths += [i[0]]
    valid_label_vectors += [i[1]]
for i in test_img_paths_and_1hotvectors:
    test_img_paths += [i[0]]
    test_label_vectors += [i[1]]
train_dataset = IOTDataset(train_img_paths, train_label_vectors, transforms=True)
valid_dataset = IOTDataset(valid_img_paths, valid_label_vectors, transforms=True)
test_dataset = IOTDataset(test_img_paths, test_label_vectors, transforms=True)

#dataloader
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

#model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = models.resnet101(pretrained=True)
for param in model.parameters():
    param.requirse_grad = True
model.fc = torch.nn.Linear(model.fc.in_features, 22)
model = model.to(device)

#loss and optimizer
loss_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#training
epochs = 50
best_accuracy = 0.0
for i in range(epochs):
    print(f'Epoch {i} / {epochs}.')
    model.train()
    running_loss, correct, total = 0, 0, 0
    for batch_data in train_dataloader:
        img_data, label_data = batch_data
        img_data, label_data = img_data.to(device), label_data.to(device)

        optimizer.zero_grad()
        outputs = model(img_data)
        loss = loss_func(outputs, label_data)

        predictions = outputs.argmax(dim=1)
        accuracy = sum(predictions == label_data) / len(label_data)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        correct += (predictions == label_data).sum().item()
        total += label_data.size(0)
    epoch_loss = running_loss / len(train_dataloader)
    epoch_accuracy = correct / total
    print(f"Train Loss: {epoch_loss:.12f}, Accuracy: {epoch_accuracy:.12f}.")

    #validation
    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0
    with torch.no_grad():
        for batch_data in valid_dataloader:
            # img_data, label_data = batch_data
            # img_data, label_data = img_data.to(device), label_data.to(device)
            # outputs = model(img_data)
            # loss = loss_func(outputs, label_data)
    
            # predictions += outputs.argmax(dim=1)
            # val_correct += (predictions == label_data).sum().item()
            # val_total += label_data.size(0)
            # val_loss += loss.item()
            img_data, label_data = batch_data
            img_data, label_data = img_data.to(device), label_data.to(device)
            outputs = model(img_data)
            loss = loss_func(outputs, label_data)

            predictions = outputs.argmax(dim=1)
            val_correct += (predictions == label_data).sum().item()
            val_total += label_data.size(0)
            val_loss += loss.item()
    
        test_loss = val_loss / len(test_dataloader)
        test_accuracy = val_correct / val_total
        print(f"Test Loss: {test_loss:.12f}, Test Accuracy: {test_accuracy:12f}")
    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy
        torch.save(model.state_dict(), f'best_model{i}.pth')
        print(f"Saved best model! Accuracy: {best_accuracy:.6f}")

print(f'Best accuracy achieved = {best_accuracy}')

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 201MB/s]  


Epoch 0 / 50.
